In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

In [2]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

In [3]:
def grayscale(a):
    return a.reshape(a.shape[0], 3, 32, 32).mean(1).reshape(a.shape[0], -1)

In [4]:
def clean(data):
    imgs = data.reshape(data.shape[0], 3, 32, 32)
    grayscale_imgs = imgs.mean(1)
    cropped_imgs = grayscale_imgs[:, 4:28, 4:28]
    img_data = cropped_imgs.reshape(data.shape[0], -1)
    img_size = np.shape(img_data)[1]
    means = np.mean(img_data, axis=1)
    meansT = means.reshape(len(means), 1)
    stds = np.std(img_data, axis=1)
    stdsT = stds.reshape(len(stds), 1)
    adj_stds = np.maximum(stdsT, 1.0 / np.sqrt(img_size))
    normalized = (img_data - meansT) / adj_stds
    return normalized

In [5]:
def read_data(directory):
    names = unpickle('{}/batches.meta'.format(directory))['label_names']
    print('names', names)
    data, labels = [], []
    for i in range(1, 6):
        filename = '{}/data_batch_{}'.format(directory, i)
        batch_data = unpickle(filename)
        if len(data) > 0:
            data = np.vstack((data, batch_data['data']))
            labels = np.hstack((labels, batch_data['labels']))
        else:
            data = batch_data['data']
            labels = batch_data['labels']
    print(np.shape(data), np.shape(labels))
    data = clean(data)
    data = data.astype(np.float32)
    return names, data, labels


In [6]:
names, data, labels = read_data('./cifar-10-batches-py')
x = tf.placeholder(tf.float32, [None, 24 * 24]) # 5000,576
y = tf.placeholder(tf.float32, [None, len(names)]) # 50000, 10
W1 = tf.Variable(tf.random_normal([5, 5, 1, 64])) # first convo layer with 64 filters size 5x5, one band greyscale, image input size 24x24
b1 = tf.Variable(tf.random_normal([64])) # biases for first convo layer
W2 = tf.Variable(tf.random_normal([5, 5, 64, 64])) # second convo layer with 64 filters size 5x5, run on the 64 outputs of first layer, image input size 12x12
b2 = tf.Variable(tf.random_normal([64])) # biases for the second convo layer
W3 = tf.Variable(tf.random_normal([6*6*64, 1024])) # fully connected layer taking output of convo layer which are 64 image inputs size 6x6 and mapping to 32x32 original image 
b3 = tf.Variable(tf.random_normal([1024])) # biases for fully connected layer
W_out = tf.Variable(tf.random_normal([1024, len(names)])) # output weights for final layer of actual labels, 1024 features  mapped to 10 labels one hot encoded
b_out = tf.Variable(tf.random_normal([len(names)])) # biases for output labels

names ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(50000, 3072) (50000,)
Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.relu(conv_with_b)
    return conv_out

In [8]:
def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1],
     padding='SAME')

In [9]:
def model():
    x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])
    conv_out1 = conv_layer(x_reshaped, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    conv_out2 = conv_layer(norm1, W2, b2)
    norm2 = tf.nn.lrn(conv_out2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    maxpool_out2 = maxpool_layer(norm2)
    maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.get_shape().as_list()[0]])
    local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
    local_out = tf.nn.relu(local)
    out = tf.add(tf.matmul(local_out, W_out), b_out)
    return out

In [10]:
model_op = model()
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model_op, labels=y))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    onehot_labels = tf.one_hot(labels, len(names), on_value=1., off_value=0., axis=-1)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(data) // 200
    print('batch size', batch_size)
    for j in tqdm(range(0, 1000)):
        #print('EPOCH', j)
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i+batch_size, :]
            batch_onehot_vals = onehot_vals[i:i+batch_size, :]
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x:batch_data, y: batch_onehot_vals})
            #if i % 1000 == 0:
            #    print(i, accuracy_val)
        #print('DONE WITH EPOCH')
        print(j, accuracy_val)
        

    saver.save(sess, './cifar10-cnn-tf1n-1000epochs.ckpt')

batch size 250


0 0.224
1 0.252
2 0.248
3 0.256
4 0.268
5 0.292
6 0.276
7 0.3
8 0.332
9 0.324
10 0.336
11 0.344
12 0.352
13 0.36


In [ ]:
def model_X(x, W1, b1, W2, b2, W3, b3, W_out, b_out):
    x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])
    conv_out1 = conv_layer(x_reshaped, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    conv_out2 = conv_layer(norm1, W2, b2)
    norm2 = tf.nn.lrn(conv_out2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    maxpool_out2 = maxpool_layer(norm2)
    maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.shape[0]])
    local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
    local_out = tf.nn.relu(local)
    out = tf.add(tf.matmul(local_out, W_out), b_out)
    return out

In [ ]:
def predict(img_data):
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, "./cifar10-cnn-tf1n-1000epochs.ckpt")
        print("Model restored.")
        W1_val = W1.eval()
        b1_val = b1.eval()
        W2_val = W2.eval()
        b2_val = b2.eval()
        W3_val = W3.eval()
        b3_val = b3.eval()
        W_out_val = W_out.eval()
        b_out_val = b_out.eval()
        model_x_out = model_X(img_data, W1_val, b1_val, W2_val, b2_val, W3_val, b3_val, W_out_val, b_out_val)
        class_num = np.argmax(model_x_out.eval(), axis=1)[0]
        class_name = names[np.argmax(model_x_out.eval(), axis=1)[0]]
        return (class_num, class_name) 

In [ ]:
class_num, class_name = predict(data[3])
print('Class Num', class_num)
print('Class', class_name)

In [ ]:
plt.figure()
plt.title(class_name)
img = np.reshape(data[3, :], (24,24))
plt.imshow(img, cmap='Greys_r')
plt.axis('off')
plt.tight_layout()

In [ ]:
# test out on test images
def read_test_data(directory):
    names = unpickle('{}/batches.meta'.format(directory))['label_names']
    print('names', names)
    data, labels = [], []
    
    filename = '{}/test_batch'.format(directory)
    batch_data = unpickle(filename)
    if len(data) > 0:
        data = np.vstack((data, batch_data['data']))
        labels = np.hstack((labels, batch_data['labels']))
    else:
        data = batch_data['data']
        labels = batch_data['labels']
    print(np.shape(data), np.shape(labels))
    data = clean(data)
    data = data.astype(np.float32)
    return names, data, labels

In [ ]:
test_names, test_data, test_labels = read_test_data('cifar-10-batches-py')

In [ ]:
test_class_num, test_class_name = predict(test_data[4])
print('Test Class Num', test_class_num)
print('Test Class Name', test_class_name)
print('Actual Class Label', test_labels[4])

In [ ]:
plt.figure()
plt.title(test_class_name)
img = np.reshape(test_data[3, :], (24,24))
plt.imshow(img, cmap='Greys_r')
plt.axis('off')
plt.tight_layout()

In [ ]:
def get_test_accuracy(test_data, test_names, test_labels):
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, "./cifar10-cnn-tf1n-1000epochs.ckpt")
        print("Model restored.")
        W1_val = W1.eval()
        b1_val = b1.eval()
        W2_val = W2.eval()
        b2_val = b2.eval()
        W3_val = W3.eval()
        b3_val = b3.eval()
        W_out_val = W_out.eval()
        b_out_val = b_out.eval()
        model_x_out = model_X(test_data, W1_val, b1_val, W2_val, b2_val, W3_val, b3_val, W_out_val, b_out_val)
        
        onehot_test_labels = tf.one_hot(test_labels, len(test_names), on_value=1., off_value=0., axis=-1)
        test_correct_pred = tf.equal(tf.argmax(model_x_out, 1), tf.argmax(onehot_test_labels, 1))
        test_accuracy = tf.reduce_mean(tf.cast(test_correct_pred, tf.float32))
        
        print('Test accuracy %f' % (test_accuracy.eval()))  
        predictions = tf.argmax(model_x_out, 1).eval()
        return (predictions, tf.cast(test_correct_pred, tf.float32).eval(), onehot_test_labels.eval())

    
predict_vals, test_correct_preds, onehot_test_lbls = get_test_accuracy(test_data, test_names, test_labels)
print(predict_vals)
print(predict_vals.shape)
print(test_correct_preds)
print(test_correct_preds.shape)
print(onehot_test_lbls.shape)

In [ ]:
outcome_test = label_binarize(test_labels, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
predictions_test = label_binarize(predict_vals, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
#print(outcome_test.shape)
#print(predictions_test.shape)
#print(outcome_test)
#print(predictions_test)
n_classes = outcome_test.shape[1]
#print(n_classes)

#fpr, tpr, thresholds = roc_curve(predictions_test, outcome_test)
#roc_auc = auc(fpr, tpr)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(outcome_test[:, i], predictions_test[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
   
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(outcome_test.ravel(), predictions_test.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for class: '+test_names[2])
plt.legend(loc="lower right")
plt.show()

# Plot ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(test_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve for CIFAR-10 CNN 1000 iter Tensorflow (area = %{0:0.2f})'.format(roc_mean))
plt.legend(loc="lower right")
plt.show()